In [4]:
# autoreload
%load_ext autoreload
%autoreload 2

# imports
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import widgets

import scienceplots
plt.style.use(['science','no-latex'])
# plt.style.use(['science'])


# change matplotlib colrormap
cmap = 'magma'
plt.rcParams['image.cmap'] = cmap

# add src to path
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(''), '../..')))
from src.dynamics import dynamics
from src.linear_response import response, response_regime_metric


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
@widgets.interact(J=(0.1, 2.0, 0.01), 
                  g=(0.1, 1, 0.01), 
                  delta_1=(0.1, 1, 0.01),
                  delta_2=(0.1, 1, 0.01),
                  a=(0.0, 1.0, 0.1), 
                  b=(0.0, 1.0, 0.1), 
                  c=(0.0, 1.0, 0.1))
def plot_determinant_sign(J, g, delta_1, delta_2, a, b, c):
    alpha_arr = np.linspace(0.1, 2.0, 100)
    beta_arr = np.linspace(0.1, 2.0, 100)

    Alpha, Beta = np.meshgrid(alpha_arr, beta_arr)

    Det = np.zeros(Alpha.shape)
    R_late = np.zeros(Alpha.shape)
    R_early = np.zeros(Alpha.shape)

    # delta = 1
    for i in range(Alpha.shape[0]):
        for j in range(Alpha.shape[1]):
            alpha = Alpha[i, j]
            beta = Beta[i, j]

            W = np.array([[J, beta*J, - delta_1*J],
                          [alpha*J, J, - delta_2*J],
                          [J, J, - g*J]])

            Det[i, j] = np.linalg.det(np.eye(3) - W)

            I = np.array([a, b, c])
            R_late[i, j], R_early[i, j] = response(W, I)

    fig, ax = plt.subplots(1, 3, figsize=(18, 5))

    im0 = ax[0].imshow(Det, extent=(0.1, 2.0, 0.1, 2.0), origin='lower')
    ax[0].set_title(r'Determinant of (I - W)')
    ax[0].set_xlabel(r'$\alpha$ (Late Inh Strength)')
    ax[0].set_ylabel(r'$\beta$ (Early Inh Strength)')
    fig.colorbar(im0, ax=ax[0])

    # plot a line where R_l = 0 on the im1 plot
    cs = ax[0].contour(alpha_arr, beta_arr, Det, levels=[0], colors='white', linewidths=1, linestyles='dashed')
    ax[0].clabel(cs, inline=True, fontsize=5, fmt=r"$R_L=0$")

    im1 = ax[1].imshow(R_late, extent=(0.1, 2.0, 0.1, 2.0), origin='lower')
    ax[1].set_title('Response of Late Excitatory Population')
    ax[1].set_xlabel(r'$\alpha$ (Late Inh Strength)')
    ax[1].set_ylabel(r'$\beta$ (Early Inh Strength)')
    fig.colorbar(im1, ax=ax[1])

    im2 = ax[2].imshow(R_early, extent=(0.1, 2.0, 0.1, 2.0), origin='lower')
    ax[2].set_title('Response of Early Excitatory Population')
    ax[2].set_xlabel(r'$\alpha$ (Late Inh Strength)')
    ax[2].set_ylabel(r'$\beta$ (Early Inh Strength)')
    fig.colorbar(im2, ax=ax[2])


interactive(children=(FloatSlider(value=1.05, description='J', max=2.0, min=0.1, step=0.01), FloatSlider(value…

In [12]:
def early_vs_late_exc(J, alpha, beta, g, delta_1, delta_2, a, b, c):

    W = np.array([[J, beta*J, - delta_1*J],
                  [alpha*J, J, - delta_2*J],
                  [J, J, - g*J]])
    
    I = np.array([a, b, c])
    R_l, R_e = response(W, I)
    return R_l, R_e
    
def plot_response_differential_exc(J, g, delta_1, delta_2, a, b, c, density=100, save_fig_location='../results/diff_exc/'):
    alpha_arr = np.linspace(0.1, 2, density)
    beta_arr = np.linspace(0.1, 2, density)

    R_l = np.zeros((len(alpha_arr), len(beta_arr)))
    R_e = np.zeros((len(alpha_arr), len(beta_arr)))

    for alpha_idx, alpha in enumerate(alpha_arr):
        for beta_idx, beta in enumerate(beta_arr):
            R_l[alpha_idx, beta_idx], R_e[alpha_idx, beta_idx] = early_vs_late_exc(J, alpha, beta, g, delta_1, delta_2, a, b, c)

    R_eff = response_regime_metric(R_l, R_e)

    fig, ax = plt.subplots(1, 2, figsize=(7, 3), dpi=250)

    vmin = min(np.min(R_l), np.min(R_e))
    vmax = max(np.max(R_l), np.max(R_e))   
    vmin = -1
    vmax = 1
    
    fs = 15
    im1 = ax[0].imshow(R_l, extent=(0.1, 2, 0.1, 2), origin='lower', aspect='auto', vmin=vmin, vmax=vmax)
    
    ax[0].set_title(r"$R_L$")
    ax[0].set_ylabel(r"$\alpha$", fontsize=fs)
    ax[0].set_xlabel(r"$\beta$", fontsize=fs)
    fig.colorbar(im1, ax=ax[0]) 

    # plot a line where R_l = 0 on the im1 plot
    cs = ax[0].contour(alpha_arr, beta_arr, R_l, levels=[0], colors='white', linewidths=1, linestyles='dashed')
    ax[0].clabel(cs, inline=True, fontsize=5, fmt=r"$R_L=0$")

    im2 = ax[1].imshow(R_e, extent=(0.1, 2, 0.1, 2), origin='lower', aspect='auto', vmin=vmin, vmax=vmax)
    ax[1].set_title(r"$R_E$")
    ax[1].set_ylabel(r"$\alpha$", fontsize=fs)
    ax[1].set_xlabel(r"$\beta$", fontsize=fs)
    fig.colorbar(im2, ax=ax[1])

    # plot a line where R_e = 0 on the im2 plot
    cs = ax[1].contour(alpha_arr, beta_arr, R_e, levels=[0], colors='white', linewidths=1, linestyles='dashed')
    ax[1].clabel(cs, inline=True, fontsize=5, fmt=r"$R_E=0$")

    fig.tight_layout()
    fig.savefig(os.path.join(save_fig_location, 'responses_differential_inh.png'), dpi=500)

    fig, ax = plt.subplots(1, 1, figsize=(4, 2.5), dpi=300)
    im = ax.imshow(R_eff, extent=(0.1, 2, 0.1, 2), origin='lower', aspect='auto')
    ax.set_title('Response Regimes')
    ax.set_ylabel(r"$\alpha$", fontsize=fs)
    ax.set_xlabel(r"$\beta$", fontsize=fs)
    cbar = fig.colorbar(im, ax=ax, ticks=[0, 1, 2])
    cbar.ax.set_yticklabels(['Acquisition', 'Early recall', 'Late recall'])
    fig.tight_layout()
    fig.savefig(os.path.join(save_fig_location, 'response_regimes_differential_inh.png'), dpi=500)
    fig.show()

In [13]:
@widgets.interact(J=widgets.FloatSlider(1., min=0, max=2.0, step=0.01), 
                  g=widgets.FloatSlider(0.6, min=0.1, max=1, step=0.01), 
                  delta_1=widgets.FloatSlider(0.3, min=0.1, max=1, step=0.01),
                  delta_2=widgets.FloatSlider(0.7, min=0.1, max=1, step=0.01),
                  a=widgets.FloatSlider(1.0, min=0.0, max=2.0, step=0.1),
                  b=widgets.FloatSlider(1.0, min=0.0, max=2.0, step=0.1), 
                  c=widgets.FloatSlider(1.0, min=0.0, max=2.0, step=0.1))
def explore_response_differential_inh(J, g, delta_1, delta_2, a, b, c):
    """ get the dynamics from the other script and plug in the values"""
    plot_response_differential_exc(J, g, delta_1, delta_2, a, b, c, density=200, save_fig_location='../../results/diff_exc_asymmetric_inh')

interactive(children=(FloatSlider(value=1.0, description='J', max=2.0, step=0.01), FloatSlider(value=0.6, desc…